<a href="https://colab.research.google.com/github/lasiadhi/Prediction-of-hypoxemia-trend/blob/master/explore_careplangeneral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#################################################
###### @author: Lasith Adhikari (python 3)  #####
############ Created on Nov 15, 2019 ############
# Code description: 
# Explore careplangeneral table to find MV patients with start and stop times

In [0]:
# Import libraries
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path
import tensorflow as tf

# Below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

# Imports for accessing Datathon data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [0]:
from google.colab import files
import seaborn as sns
import pickle
import matplotlib.pyplot as plt 

In [0]:
auth.authenticate_user()

In [0]:
# Note that this should be the project for the datathon work,
# not the physionet-data project which is for data hosting.
project_id = 'hst-953-2019'
os.environ['GOOGLE_CLOUD_PROJECT'] = project_id

In [0]:

# Read data from BigQuery into pandas dataframes.
def run_query(query):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

In [0]:
df_careplan = run_query("""
select *
from `physionet-data.eicu_crd.careplangeneral` 
"""
)

In [7]:
df_careplan.head()

,cplgeneralid,patientunitstayid,activeupondischarge,cplitemoffset,cplgroup,cplitemvalue
0,3734900,141168,False,2160,Airway,Not intubated/partial airway obstruction
1,4023241,141244,False,401,DVT Prophylaxis,Not indicated - Patient ambulating
2,3720578,141296,False,35,Airway,Not intubated/partial airway obstruction
3,3995436,141313,False,37,Ventilation,Ventilated - rapid wean/extubation
4,3445427,141451,False,52,Ventilation,Ventilated - rapid wean/extubation


In [14]:

df_careplan.shape

(3115018, 6)

In [15]:
df_careplan.loc[df_careplan.cplitemvalue.str.contains('Ventilated'),'patientunitstayid'].nunique()

50490

In [10]:
df_careplan.loc[df_careplan.cplgroup=='Ventilation','patientunitstayid'].nunique()

188142

In [11]:
df_careplan.patientunitstayid.nunique()

197396

In [0]:
df_careplan.cplitemoffset = round(df_careplan.cplitemoffset/60.0,2) 

In [16]:
#examples:
df_careplan[df_careplan.patientunitstayid==588232].sort_values(by='cplitemoffset')

#{'ICU Admit': [0.0],
# 'ICU Discharge': [173.51666666666668],
# 'IV': [(-0.017, 19.067), (71.983, 135.15)],
# 'NIV': [(71.15, 71.983), (135.15, 150.05)],
# 'O2': [(19.067, nan), (150.05, nan)]}

,cplgeneralid,patientunitstayid,activeupondischarge,cplitemoffset,cplgroup,cplitemvalue
340598,12622116,588232,False,0.00,DVT Prophylaxis,
340600,15677399,588232,False,0.00,Airway,
340601,10608952,588232,False,0.00,Stress Ulcer Prophylaxis,
340602,14964944,588232,False,0.00,Care Providers,
340603,11258187,588232,False,0.00,Ventilation,
480573,10800547,588232,False,0.00,Acuity,High
2398063,13671065,588232,True,1.20,Stress Ulcer Prophylaxis,H-2 antagonists
1479886,11834761,588232,False,1.20,Ventilation,Ventilated - with daily extubation evaluation
1025850,11199731,588232,False,1.20,Airway,Intubated/oral ETT
2132690,14311125,588232,True,4.27,Care Limitation,Full therapy


In [0]:
df_careplan[(df_careplan.patientunitstayid==588232) & ((df_careplan.cplgroup=='Ventilation') | (df_careplan.cplgroup=='Airway') )].sort_values(by='cplitemoffset')

,cplgeneralid,patientunitstayid,activeupondischarge,cplitemoffset,cplgroup,cplitemvalue
340600,15677399,588232,False,0,Airway,
340603,11258187,588232,False,0,Ventilation,
1025850,11199731,588232,False,72,Airway,Intubated/oral ETT
1479886,11834761,588232,False,72,Ventilation,Ventilated - with daily extubation evaluation
1218123,13429487,588232,False,1124,Ventilation,Spontaneous - adequate
1345768,13861464,588232,False,1124,Airway,Not intubated/normal airway
1025849,15425704,588232,False,4571,Airway,Intubated/oral ETT
1479885,11023533,588232,False,4571,Ventilation,Ventilated - with daily extubation evaluation
1174606,14294054,588232,False,8217,Ventilation,Spontaneous - tenuous
2911439,14868887,588232,True,8217,Airway,Not intubated/normal airway


In [0]:
#examples:
df_careplan[df_careplan.patientunitstayid==587672].sort_values(by='cplitemoffset')

#{'ICU Admit': [0.0],
# 'ICU Discharge': [156.05],
# 'IV': [(17.267, 109.517)],
# 'NIV': [],
# 'O2': [(-0.733, 17.267), (109.517, 146.267)]}

,cplgeneralid,patientunitstayid,activeupondischarge,cplitemoffset,cplgroup,cplitemvalue
340580,12611449,587672,False,0.02,Ventilation,
340581,13694945,587672,False,0.02,Airway,
340582,10591524,587672,False,0.02,Stress Ulcer Prophylaxis,
340583,13828727,587672,False,0.02,DVT Prophylaxis,
340584,12497699,587672,False,0.02,Care Providers,
1872827,11876094,587672,True,0.02,Acuity,High
2398053,13087900,587672,True,0.15,Stress Ulcer Prophylaxis,H-2 antagonists
2132671,11501446,587672,True,0.15,Care Limitation,Full therapy
1345761,14285172,587672,False,0.15,Airway,Not intubated/normal airway
1218112,10770792,587672,False,0.15,Ventilation,Spontaneous - adequate


In [0]:
#examples:
df_careplan[df_careplan.patientunitstayid==588720].sort_values(by='cplitemoffset')
#{'ICU Admit': [0.0],
# 'ICU Discharge': [408.3833333333333],
# 'IV': [(-0.35, 218.15), (316.4, 334.15)],
# 'NIV': [],
# 'O2': [(334.15, 374.15)]}

,cplgeneralid,patientunitstayid,activeupondischarge,cplitemoffset,cplgroup,cplitemvalue
42238,13596377,588720,False,0.30,Ventilation,Ventilated - with no daily extubation trial
682798,11158499,588720,False,0.30,Care Limitation,Full therapy
1025856,13668778,588720,False,0.30,Airway,Intubated/oral ETT
929649,11335941,588720,False,0.35,Stress Ulcer Prophylaxis,H-2 antagonists
1077850,12969677,588720,False,0.35,DVT Prophylaxis,Compression devices
1479892,12279204,588720,False,31.23,Ventilation,Ventilated - with daily extubation evaluation
1345778,11416009,588720,False,218.75,Airway,Not intubated/normal airway
1218132,12768531,588720,False,218.75,Ventilation,Spontaneous - adequate
1345779,10868908,588720,False,233.92,Airway,Not intubated/normal airway
1218135,10958279,588720,False,233.92,Ventilation,Spontaneous - adequate


In [0]:
#examples:
df_careplan[df_careplan.patientunitstayid==589520].sort_values(by='cplitemoffset')
# no IV episod involved here

,cplgeneralid,patientunitstayid,activeupondischarge,cplitemoffset,cplgroup,cplitemvalue
2132730,11707610,589520,True,0.32,Care Limitation,Full therapy
2713871,11054814,589520,True,0.32,Ventilation,Spontaneous - adequate
2911483,15701434,589520,True,0.32,Airway,Not intubated/normal airway
2398086,10818134,589520,True,5.47,Stress Ulcer Prophylaxis,H-2 antagonists
2523683,14540518,589520,True,5.47,DVT Prophylaxis,Compression devices


In [0]:
#examples:
df_careplan[df_careplan.patientunitstayid==591046].sort_values(by='cplitemoffset')
#591046: {'ICU Admit': [0.0],
#  'ICU Discharge': [40.6],
#  'IV': [(-0.033, 19.083)],
#  'NIV': [(19.083, 38.7)],
#  'O2': [(38.7, nan)]},#

,cplgeneralid,patientunitstayid,activeupondischarge,cplitemoffset,cplgroup,cplitemvalue
682868,11533264,591046,False,0.28,Care Limitation,Full therapy
929699,12336220,591046,False,0.28,Stress Ulcer Prophylaxis,H-2 antagonists
1025891,15021075,591046,False,0.28,Airway,Intubated/oral ETT
1077888,10857835,591046,False,0.28,DVT Prophylaxis,Compression devices
1479931,10667971,591046,False,0.28,Ventilation,Ventilated - with daily extubation evaluation
1218202,11858550,591046,False,23.63,Ventilation,Spontaneous - adequate
1345858,15257531,591046,False,23.63,Airway,Not intubated/normal airway


In [0]:
df_careplan[df_careplan.patientunitstayid==591526].sort_values(by='cplitemoffset')
# seems no IV epi. But this pts was in IV according to the mech_vent tble.

,cplgeneralid,patientunitstayid,activeupondischarge,cplitemoffset,cplgroup,cplitemvalue
682873,12864696,591526,False,0.17,Care Limitation,Full therapy
929704,12951995,591526,False,0.17,Stress Ulcer Prophylaxis,H-2 antagonists
1218208,12726453,591526,False,0.17,Ventilation,Spontaneous - adequate
1345865,13791371,591526,False,0.17,Airway,Not intubated/normal airway
1077889,12394008,591526,False,12.88,DVT Prophylaxis,Compression devices
1410271,10845861,591526,False,19.18,DVT Prophylaxis,Combined device and drug therapy


In [0]:
df_careplan[df_careplan.patientunitstayid==592156].sort_values(by='cplitemoffset')
#592156: {'ICU Admit': [0.0],
#  'ICU Discharge': [68.13333333333334],
#  'IV': [(-0.183, 5.283)],
#  'NIV': [(5.283, 18.983)],
#  'O2': [(18.983, nan)]},

,cplgeneralid,patientunitstayid,activeupondischarge,cplitemoffset,cplgroup,cplitemvalue
42279,12843522,592156,False,0.37,Ventilation,Ventilated - rapid wean/extubation
1025905,12652940,592156,False,0.37,Airway,Intubated/oral ETT
1077911,12749153,592156,False,0.37,DVT Prophylaxis,Compression devices
2132819,15681484,592156,True,0.37,Care Limitation,Full therapy
2398130,15567235,592156,True,0.37,Stress Ulcer Prophylaxis,H-2 antagonists
1313358,14900256,592156,False,18.17,Ventilation,Non-invasive ventilation
2911575,10935040,592156,True,18.17,Airway,Not intubated/normal airway
1174648,14019581,592156,False,30.87,Ventilation,Spontaneous - tenuous
3052311,11468468,592156,True,36.45,DVT Prophylaxis,Combined device and drug therapy
2713951,10510149,592156,True,66.02,Ventilation,Spontaneous - adequate


In [0]:
df_careplan[df_careplan.patientunitstayid==592338].sort_values(by='cplitemoffset')
#592338: {'ICU Admit': [0.0],
#  'ICU Discharge': [66.38333333333334],
#  'IV': [(-0.233, 25.267)],
#  'NIV': [],
#  'O2': [(25.267, nan)]},

,cplgeneralid,patientunitstayid,activeupondischarge,cplitemoffset,cplgroup,cplitemvalue
42281,12190116,592338,False,0.28,Ventilation,Ventilated - rapid wean/extubation
605447,11160817,592338,False,0.28,Activity,Bedrest
1025906,15093539,592338,False,0.28,Airway,Intubated/oral ETT
2132826,12628132,592338,True,0.28,Care Limitation,Full therapy
2523711,15687908,592338,True,0.28,DVT Prophylaxis,Compression devices
1781304,11684614,592338,True,7.43,Activity,
2398133,12019671,592338,True,7.43,Stress Ulcer Prophylaxis,H-2 antagonists
2713958,14044600,592338,True,24.48,Ventilation,Spontaneous - adequate
2911582,12131659,592338,True,24.48,Airway,Not intubated/normal airway


In [0]:
df_careplan[df_careplan.patientunitstayid==593127].sort_values(by='cplitemoffset')
# DNR patient. IV end missing

,cplgeneralid,patientunitstayid,activeupondischarge,cplitemoffset,cplgroup,cplitemvalue
682935,13697064,593127,False,0.15,Care Limitation,Full therapy
1174658,12987778,593127,False,0.15,Ventilation,Spontaneous - tenuous
1345942,12953809,593127,False,0.15,Airway,Not intubated/normal airway
2523723,12764423,593127,True,0.15,DVT Prophylaxis,Compression devices
2507645,11707884,593127,True,2.90,Airway,Intubated/oral ETT
2584920,15434264,593127,True,2.90,Stress Ulcer Prophylaxis,Proton pump inhibitor
3108929,13999673,593127,True,2.90,Ventilation,Ventilated - with daily extubation evaluation
2476801,14621760,593127,True,20.80,Care Limitation,Do not resuscitate


In [0]:
df_careplan[df_careplan.patientunitstayid==581995].sort_values(by='cplitemoffset')


,cplgeneralid,patientunitstayid,activeupondischarge,cplitemoffset,cplgroup,cplitemvalue
2646844,14173405,581995,True,0.05,Ventilation,Spontaneous - tenuous
2911235,11613830,581995,True,0.05,Airway,Not intubated/normal airway
2476743,12580133,581995,True,2.60,Care Limitation,Do not resuscitate


In [0]:
df_careplan[df_careplan.patientunitstayid==784786].sort_values(by='cplitemoffset')


,cplgeneralid,patientunitstayid,activeupondischarge,cplitemoffset,cplgroup,cplitemvalue
1482522,13294185,784786,False,0.15,Ventilation,Ventilated - with daily extubation evaluation
644259,14561486,784786,False,0.15,DVT Prophylaxis,Drug therapy
1128321,11612310,784786,False,0.15,Stress Ulcer Prophylaxis,Proton pump inhibitor
687346,14839237,784786,False,0.15,Care Limitation,Full therapy
1028618,14253043,784786,False,0.15,Airway,Intubated/oral ETT
1351307,11955943,784786,False,11.55,Airway,Not intubated/normal airway
1223437,14026168,784786,False,11.55,Ventilation,Spontaneous - adequate
1482521,10449764,784786,False,18.30,Ventilation,Ventilated - with daily extubation evaluation
1028617,15146195,784786,False,18.30,Airway,Intubated/oral ETT
1176608,11593466,784786,False,63.80,Ventilation,Spontaneous - tenuous


In [0]:

# pts with cplgroup == ventilation
df_careplan[df_careplan.cplgroup=='Ventilation']['patientunitstayid'].nunique()

188142

In [0]:
# find all cplitemvalues under Ventilation
df_careplan.loc[df_careplan.cplgroup=='Ventilation','cplitemvalue'].unique()

array(['Ventilated - rapid wean/extubation',
       'Ventilated - with no daily extubation trial',
       'Ventilated - chronic dependency', '', 'Spontaneous - tenuous',
       'Spontaneous - adequate', 'Non-invasive ventilation',
       'Ventilated - with daily extubation evaluation'], dtype=object)

In [0]:
df_careplan.loc[df_careplan.cplgroup=='Care Limitation','cplitemvalue'].unique()

array(['No CPR', 'No augmentation of care', 'No intubation',
       'Comfort measures only', 'No blood products', 'No cardioversion',
       'No blood draws', 'No vasopressors/inotropes',
       'Advance directives', 'Full therapy', 'Do not resuscitate'],
      dtype=object)

In [0]:
df_careplan[df_careplan.cplitemvalue=='Ventilated - chronic dependency']

,cplgeneralid,patientunitstayid,activeupondischarge,cplitemoffset,cplgroup,cplitemvalue
241,3622362,152518,False,48.70,Ventilation,Ventilated - chronic dependency
294,3876770,154945,False,103.97,Ventilation,Ventilated - chronic dependency
450,3733309,160515,False,2.85,Ventilation,Ventilated - chronic dependency
631,3976779,167641,False,8.13,Ventilation,Ventilated - chronic dependency
652,3501274,168648,False,0.15,Ventilation,Ventilated - chronic dependency
...,...,...,...,...,...,...
1775617,57041264,3245379,True,2.78,Ventilation,Ventilated - chronic dependency
1776448,58858645,3332462,True,601.12,Ventilation,Ventilated - chronic dependency
1776869,59078762,3335872,True,1.98,Ventilation,Ventilated - chronic dependency
1777918,59208717,3345335,True,1.20,Ventilation,Ventilated - chronic dependency


In [0]:
dff = df_careplan[df_careplan.patientunitstayid==3332462].sort_values(by='cplitemoffset').reset_index(drop=True)
dff = dff[dff.cplgroup =='Ventilation']

In [0]:
df_careplan[df_careplan.patientunitstayid==3332462].sort_values(by='cplitemoffset').reset_index(drop=True)

,cplgeneralid,patientunitstayid,activeupondischarge,cplitemoffset,cplgroup,cplitemvalue
0,59057910,3332462,False,2.80,Stress Ulcer Prophylaxis,Proton pump inhibitor
1,59007620,3332462,False,2.80,Care Limitation,Full therapy
2,58959100,3332462,False,2.80,DVT Prophylaxis,Compression devices
3,58892724,3332462,False,2.80,Activity,HOB 30 degrees
4,58980711,3332462,False,2.80,Ventilation,Ventilated - chronic dependency
...,...,...,...,...,...,...
79,58974937,3332462,True,601.12,Airway,Intubated/trach-chronic
80,59045445,3332462,True,601.12,Activity,HOB 30 degrees
81,58865446,3332462,True,601.12,Sedation,Intermittent bolus
82,59007783,3332462,True,601.12,DVT Prophylaxis,Compression devices


In [0]:
dff

,cplgeneralid,patientunitstayid,activeupondischarge,cplitemoffset,cplgroup,cplitemvalue
4,58980711,3332462,False,2.80,Ventilation,Ventilated - chronic dependency
30,58949014,3332462,False,146.43,Ventilation,Ventilated - chronic dependency
67,58858232,3332462,False,485.78,Ventilation,Ventilated - chronic dependency
78,58858645,3332462,True,601.12,Ventilation,Ventilated - chronic dependency


**Let's extract MV eps - create a dict**

In [0]:
# example: 
df_pt_ = df_careplan[df_careplan.patientunitstayid==588720].sort_values(by='cplitemoffset')
#{'ICU Admit': [0.0],
# 'ICU Discharge': [408.3833333333333],
# 'IV': [(-0.35, 218.15), (316.4, 334.15)],
# 'NIV': [],
# 'O2': [(334.15, 374.15)]}

In [58]:
df_pt_ = df_careplan[df_careplan.patientunitstayid==563292].sort_values(by='cplitemoffset')
df_pt_

,cplgeneralid,patientunitstayid,activeupondischarge,cplitemoffset,cplgroup,cplitemvalue
682264,12819863,563292,False,0.32,Care Limitation,Full therapy
929251,12287779,563292,False,0.32,Stress Ulcer Prophylaxis,H-2 antagonists
1025490,10995457,563292,False,0.32,Airway,Intubated/oral ETT
1479548,11542916,563292,False,0.32,Ventilation,Ventilated - with daily extubation evaluation
2523324,12502793,563292,True,0.32,DVT Prophylaxis,Compression devices
1000148,11110371,563292,False,13.13,Care Limitation,Do not resuscitate
2646765,15636849,563292,True,75.70,Ventilation,Spontaneous - tenuous
2910562,11811071,563292,True,75.70,Airway,Not intubated/normal airway
1567987,12246529,563292,True,75.75,Care Limitation,Comfort measures only
2476654,12246530,563292,True,75.75,Care Limitation,Do not resuscitate


In [50]:
df_pt_ = df_careplan[df_careplan.patientunitstayid==542950].sort_values(by='cplitemoffset')
df_pt_

,cplgeneralid,patientunitstayid,activeupondischarge,cplitemoffset,cplgroup,cplitemvalue
2073917,10874257,542950,True,0.4,DVT Prophylaxis,Drug therapy
2131224,15000809,542950,True,0.4,Care Limitation,Full therapy
2507579,13005193,542950,True,0.4,Airway,Intubated/oral ETT
3108854,13333255,542950,True,0.4,Ventilation,Ventilated - with daily extubation evaluation


In [51]:
#df_pt_ = df_pt_[(df_pt_.cplgroup=='Ventilation') |\
#                ((df_pt_.cplgroup=='Care Limitation') & (df_pt_.cplitemvalue=='Do not resuscitate')) |\
#                ((df_pt_.cplgroup=='Care Limitation') & (df_pt_.cplitemvalue=='No intubation'))]
df_pt_ = df_pt_[(df_pt_.cplgroup=='Ventilation')]
df_pt_ 
# It seems we do not need to check for Care limitations -- patient may get continous vent even after those care limitations 

,cplgeneralid,patientunitstayid,activeupondischarge,cplitemoffset,cplgroup,cplitemvalue
3108854,13333255,542950,True,0.4,Ventilation,Ventilated - with daily extubation evaluation


In [52]:
df_pt_['vent_start'] = df_pt_.cplitemvalue.apply(lambda x: 'start' if 'Ventilated' in str(x) else 'end')
#df_pt_['vent_end'] = df_pt_.cplitemvalue.apply(lambda x: '- 'if 'Ventilated' in str(x) else 'end')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [53]:
df_pt_

,cplgeneralid,patientunitstayid,activeupondischarge,cplitemoffset,cplgroup,cplitemvalue,vent_start
3108854,13333255,542950,True,0.4,Ventilation,Ventilated - with daily extubation evaluation,start


In [0]:
# new version for one icustay
event_dict = {}
event_dict['IV'] = []
first_start_occurance = 0
IV_start = np.nan
IV_end = np.nan

for index, row  in df_pt_.iterrows():
   if (row['vent_start'] == 'start') & (first_start_occurance == 0):
     IV_start = round(row['cplitemoffset'],2)
     first_start_occurance = 1
   elif (row['vent_start'] == 'end') & (first_start_occurance == 1):
     IV_end = round(row['cplitemoffset'],2)
     event_dict['IV'].append((IV_start,IV_end))
     first_start_occurance = 0
  
if (first_start_occurance == 1):
  event_dict['IV'].append((IV_start,np.nan))

In [55]:
event_dict

{'IV': [(0.4, nan)]}

In [0]:
# let's run this rule for all patients in the generalcareplan:

In [56]:
# new version
# Ventilated start event should be available. Otherwise VI = []
event_icu_dict = {}
i = 1
for pid in df_careplan.patientunitstayid.unique():

  if i%1000==0:
    print(i)
  df_pt_ = df_careplan[df_careplan.patientunitstayid==pid].sort_values(by='cplitemoffset')
  
  df_pt_ = df_pt_[(df_pt_.cplgroup=='Ventilation')]
  df_pt_['vent_start'] = df_pt_.cplitemvalue.apply(lambda x: 'start' if 'Ventilated' in str(x) else 'end')

  event_dict = {}
  event_dict['IV'] = []
  first_start_occurance = 0

  for index, row  in df_pt_.iterrows():
    if (row['vent_start'] == 'start') & (first_start_occurance == 0):
      IV_start = round(row['cplitemoffset'],2)
      first_start_occurance = 1
    elif (row['vent_start'] == 'end') & (first_start_occurance == 1):
      IV_end = round(row['cplitemoffset'],2)
      event_dict['IV'].append((IV_start,IV_end))
      first_start_occurance = 0
    
  if (first_start_occurance == 1):
    event_dict['IV'].append((IV_start,np.nan))

  event_icu_dict[pid] =  event_dict
  i+=1

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
159000
1

In [57]:
event_icu_dict[563292]

{'IV': [(0.32, 75.7)]}

In [0]:
f = open("MVdict_from_careplane_ver1.pkl","wb")
pickle.dump(event_icu_dict,f)
f.close()
#files.download('MVdict_from_careplane_ver1.pkl')

## Append ICU discharge times

In [0]:
# read patients data
df_patients = run_query("""
select patientunitstayid,unitDischargeOffset
from `physionet-data.eicu_crd.patient` 
"""
)

In [64]:
df_patients.head()

,patientunitstayid,unitDischargeOffset
0,1137569,9246
1,486998,254
2,497557,6
3,1218211,844
4,2188050,1119


In [66]:
df_patients.patientunitstayid.nunique()

200859

In [67]:
df_patients.shape

(200859, 2)

In [0]:
df_patients.unitDischargeOffset = df_patients.unitDischargeOffset/60.0

In [0]:
df_patients.unitDischargeOffset = round(df_patients.unitDischargeOffset,2)

In [71]:
df_patients

,patientunitstayid,unitDischargeOffset
0,1137569,154.10
1,486998,4.23
2,497557,0.10
3,1218211,14.07
4,2188050,18.65
...,...,...
200854,327100,0.00
200855,3001917,0.00
200856,3008375,0.00
200857,3014158,0.00


In [91]:
df_patients.loc[df_patients.patientunitstayid==1137569,'unitDischargeOffset'].values[0]

154.1

In [0]:
event_icu_dict_pts = event_icu_dict.copy()

In [0]:
for id in event_icu_dict.keys():
  event_icu_dict_pts[id]['ICU Admit'] = [0]
  event_icu_dict_pts[id]['ICU Discharge'] = [df_patients.loc[df_patients.patientunitstayid==id,'unitDischargeOffset'].values[0]]

In [0]:
# example: 
df_pt_ = df_careplan[df_careplan.patientunitstayid==588720].sort_values(by='cplitemoffset')
#{'ICU Admit': [0.0],
# 'ICU Discharge': [408.3833333333333],
# 'IV': [(-0.35, 218.15), (316.4, 334.15)],
# 'NIV': [],
# 'O2': [(334.15, 374.15)]}

In [102]:
event_icu_dict_pts[588720]

{'ICU Admit': [0],
 'ICU Discharge': [408.38],
 'IV': [(0.3, 218.75), (319.9, 335.1)]}

In [0]:
f = open("MVdict_from_careplane_with_DC.pkl","wb")
pickle.dump(event_icu_dict_pts,f)
f.close()
#files.download('MVdict_from_careplane_with_DC.pkl')

# Old codes

In [0]:
event_dict = {}
#event_dict['ICU Admit'] = np.nan # fill later 
#event_dict['ICU Discharge']  = np.nan # fill later 
event_dict['IV'] = []
first_start_occurance = 0
first_end_occurance = 0
IV_start = np.nan
IV_end = np.nan

for index, row  in df_pt_.iterrows():
   if (row['vent_start'] == 'start') & (first_start_occurance == 0):
     IV_start = round(row['cplitemoffset'],2)
     first_start_occurance = 1
   elif (row['vent_start'] == 'end') & (first_start_occurance == 1) & (first_end_occurance == 0):
     IV_end = round(row['cplitemoffset'],2)
     first_end_occurance = 1
     if (first_start_occurance==1) & (first_end_occurance==1):
       event_dict['IV'].append((IV_start,IV_end))
       first_start_occurance = 0
       first_end_occurance = 0
  
if (first_start_occurance == 1) & (first_end_occurance == 0):
  event_dict['IV'].append((IV_start,np.nan))
#elif (first_start_occurance == 0) & (first_end_occurance == 1):
#  event_dict['IV'].append((np.nan,IV_end))

In [0]:
# old version
event_icu_dict = {}
i = 1
for pid in df_careplan.patientunitstayid.unique():

  if i%1000==0:
    print(i)
  df_pt_ = df_careplan[df_careplan.patientunitstayid==pid].sort_values(by='cplitemoffset')
  
  df_pt_ = df_pt_[(df_pt_.cplgroup=='Ventilation')]
  df_pt_['vent_start'] = df_pt_.cplitemvalue.apply(lambda x: 'start' if 'Ventilated' in str(x) else 'end')

  event_dict = {}
  event_dict['ICU Admit'] = np.nan # fill later 
  event_dict['ICU Discharge']  = np.nan # fill later 
  event_dict['IV'] = []
  first_start_occurance = 0
  first_end_occurance = 0

  for index, row  in df_pt_.iterrows():
    if (row['vent_start'] == 'start') & (first_start_occurance == 0):
      IV_start = round(row['cplitemoffset'],2)
      first_start_occurance = 1
    elif (row['vent_start'] == 'end') & (first_start_occurance == 1) & (first_end_occurance == 0):
      IV_end = round(row['cplitemoffset'],2)
      first_end_occurance = 1
      if (first_start_occurance==1) & (first_end_occurance==1):
        event_dict['IV'].append((IV_start,IV_end))
        first_start_occurance = 0
        first_end_occurance = 0
    
  if (first_start_occurance == 1) & (first_end_occurance == 0):
    event_dict['IV'].append((IV_start,np.nan))

  event_icu_dict[pid] =  event_dict
  i+=1
  

In [154]:
event_icu_dict[563292]

{'ICU Admit': nan, 'ICU Discharge': nan, 'IV': [(0.32, 13.13)]}

In [0]:
f = open("MVdict_from_careplane.pkl","wb")
pickle.dump(event_icu_dict,f)
f.close()
#files.download('MVdict_from_careplane.pkl')